# Instructions

+ Rename this file to `<your nusnetid>.ipynb`. For example, `e0321286.ipynb`.
+ The completed assignment must be uploaded to LumiNUS (Files > Assignment Submissions > PA3)
+ Submission deadline: 11AM, 9th October
+ Late Submission Rules: (1) 10 points will be deducted if submitted after 11 AM, 9th October and before 11 AM, 10th October; (2) Submissions after 11 AM, 10th October will not be graded and will be given **zero** marks.

+ This assignment is of 20 marks total. There are two sections I and II.
+ The  <font color='purple'> marks </font> associated with each question is mentioned at the end of the question in  <font color='purple'> purple </font> color.
+ Questions with <font color='blue'> blue </font> color are not graded, but you are encouraged to answer them for your better understanding.

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import warnings
warnings.filterwarnings("ignore")


## I. Understanding Grid Search and Pipelines


**Q1) Reimplement lines 2-7 of the code below by making all of the following modifications.** <font color='purple'> (2) </font> 

(a) Use sklearn's KFold() and find the best estimator using AUC scores obtained in each cross-validation step.

(b) Do NOT use GridSearchCV(). You can use Pipeline(). 

Select the best combination of hyperparameters based on 3-fold cross-validation scores. 

Print the average AUC score for each cross-validation fold.

In [2]:
# #1
# X,Y = make_classification(n_samples=100, n_features=20, n_informative=10, n_redundant=10, n_classes=2, n_clusters_per_class=1, shift=2,scale=None, random_state=0)
# #2
# parameters = dict(pca__n_components=[5,8],lr__C=[1, 3])
# #3
# estA = Pipeline(steps=[('pca', PCA()), ('lr', LogisticRegression())])
# #4
# estimator_A = GridSearchCV(estA, parameters, cv=3, scoring = 'roc_auc')
# #5
# estimator_A.fit(X,Y)
# #6
# A = estimator_A.best_estimator_
# #7
# print(estimator_A.best_params_)




X,Y = make_classification(n_samples=100, n_features=20, n_informative=10, n_redundant=10, n_classes=2, n_clusters_per_class=1, shift=2,scale=None, random_state=0)
parameters = dict(pca__n_components=[5,8],lr__C=[1, 3])
estA = Pipeline(steps=[('pca', PCA()), ('lr', LogisticRegression())])


X_train = []
X_test = []
Y_train = []
Y_test = []
best = ''
score = 0
seed = 0
kfold = KFold(n_splits=3, random_state=seed)

for train_index, test_index in kfold.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]


for pca in parameters["pca__n_components"]:
    for lr in parameters["lr__C"]:
        estA.set_params(pca__n_components = pca)
        estA.set_params(lr__C = lr)
        scores = cross_val_score(estA, X_test, Y_test, cv = 3, scoring="roc_auc")
        if (scores.mean() > score):
            score = scores.mean()
            best = 'pca: ' + str(pca) + " " + "lr: " + str(lr)
    print(scores.mean())
    
    
print(best)

0.6555555555555556
0.7777777777777777
pca: 8 lr: 1


**Q2) Reimplement lines 2 and 3 of the code below by making all of the following modifications.** <font color='purple'> (2) </font> 

(a) Use fit() and transform() for each operation in the pipeline

(b) Use KFold() and find the mean and standard deviation of AUC scores obtained over the 3 folds

(c) Do NOT use the Pipeline() object or cross_val_score() function

Ensure that there is no data leakage. 

Print the AUC score in each fold and the mean and standard deviation of AUC scores over the 3 folds.

In [42]:
#1
X,Y = make_classification(n_samples=100, n_features=20, n_informative=10, n_redundant=10, n_classes=2, n_clusters_per_class=1, shift=2,scale=None, random_state=0)
#2
estA = Pipeline(steps=[('pca', PCA()), ('lr', LogisticRegression())])
#3
scores = cross_val_score(estA, X, Y, cv=3, scoring='roc_auc')
#4
print(scores, np.mean(scores), np.std(scores))


from sklearn.metrics import precision_score, recall_score, roc_curve, auc

X,Y = make_classification(n_samples=100, n_features=20, n_informative=10, n_redundant=10, n_classes=2, n_clusters_per_class=1, shift=2,scale=None, random_state=0)
kfold = KFold(n_splits=3)
auc = []
for train_index, test_index in kfold.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    pca = PCA()
    lr = LogisticRegression()
    pca.fit(X_train)
    pca.transform(X_train)
    lr.fit(X_train, Y_train)
    #y_pred = lr.predict(X_test)  
#     roc_auc = roc_auc_score(Y_test, y_pred)
#     auc.append(roc_auc)

    roc_auc=roc_auc_score(Y_test, lr.predict_proba(X_test)[:,1])
    auc.append(roc_auc)


        
print(auc, np.mean(auc), np.std(auc))



[0.98269896 0.90441176 0.96323529] 0.9501153402537486 0.03327983578332014
[0.9722222222222222, 0.8685185185185185, 0.9407407407407408] 0.9271604938271604 0.04341222462349938


**Q3) Evaluate the performance of the model from question 2 (trained on X,Y) on the test set (X2) generated below.**

Use metrics precision, recall, sensitivity, specificity and AUC. <font color='purple'> (0.5) </font> 

In [4]:
X2,Y2 = make_classification(n_samples=500, n_features=20, n_informative=10, n_redundant=10, n_classes=2, n_clusters_per_class=1, shift=2,scale=None, random_state=0)

from sklearn.metrics import confusion_matrix

Y_pred = lr.predict(X2)
print('Precision:', precision_score(Y2, Y_pred))
print('Recall:', recall_score(Y2, Y_pred))
#print('AUC:', roc_auc_score(Y2, y_pred))
# false_positive_rate, true_positive_rate, thresholds = roc_curve(Y2, y_pred)
# roc_auc = auc(false_positive_rate, true_positive_rate)
# print('AUC:', roc_auc)


cm1 = confusion_matrix(Y2, Y_pred)
#print('Confusion Matrix : \n', cm1)

sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)


Y_pred_prob = lr.predict_proba(X2)[:, 1]
fpr1, tpr1, thresholds1 = roc_curve(Y2, Y_pred_prob)


auc = cross_val_score(lr, X2, Y2, cv=3, scoring='roc_auc')
print("AUC scores: ", auc)

# #def plot_roc(fpr, tpr):
# plt.figure()
# roc_auc = auc(fpr1, tpr1)
# plt.plot(fpr1, tpr1, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
# plt.title('ROC'); plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
# plt.legend(loc="lower right")
# plt.show()

# #plot_roc(fpr1,tpr1)

Precision: 0.5463659147869674
Recall: 0.8650793650793651
Sensitivity :  0.2701612903225806
Specificity :  0.8650793650793651
AUC scores:  [0.97776822 0.96213425 0.95281649]


## II. Kaggle Competition
In this section, we will attempt a relatively simple Kaggle competition: Porto Seguro's Safe Driver Prediction. The aim is to predict if a driver will file an insurance claim next year. Please read the description of the challenge on [Kaggle](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction). The competition is closed and we cannot participate in it. So, we'll just use the training data provided in this assignment.  

Create a Kaggle account (if you don't have one) and download `train.csv` from the Data tab on the competition webpage. Read the Data description carefully. We'll build and evaluate end-to-end machine learning pipelines using sklearn functions. 

**Q4) Data Loading** <font color='purple'> (2) </font> 

(i) Load the data into a dataframe `df`, and print the number of observations (rows). This is a big dataset, so use the following code to sample only 1% of the observations using the code:`df = df.sample(frac=0.01, random_state=0)`. </font>  <font color='purple'> (0.5) </font> 

Use this sampled data for all the remaining questions in this section. **Make sure you set `random_state=0`, so your code can be reproduced exactly.** Then separate the target variable `Y` from the predictors `X`. Note that the `id` is not a predictor. <font color='blue'> Why? 

   
(ii) Print the number of classes in the target variable, and number of observations available in each class. <font color='purple'> (0.5) </font> 
    
(iii) Split the data for training (80%) (`X_train, Y_train`) and testing (20%) (`X_test, Y_test`). Print the number of observations in the training set. <font color='purple'> (0.5) </font> 
    
(iv) The features in this dataset are heterogeneous. Look at the column names and make separate lists of names for categorical (nominal) features `cat_feat`, binary features `bin_feat`, and numerical features (continuous and ordinal features) `num_feat`. Note that, in this analysis, we are considering binary features separately, not as part of categorical or numerical. Print the number of features in each of these 3 lists. <font color='purple'> (0.5) </font> 

In [5]:
df = pd.read_csv('train.csv')
df = df.sample(frac=0.01, random_state=0)


In [6]:
#print(len(df.index))
#print(df.dtypes)
Y = df['target']
columns = ['target', 'id']
X = df.drop(columns, axis = 1)
print(Y.shape)
print('number of classes in target variable ', len(Y.unique()))
print('number of observations:\n', Y.value_counts())


#print(X)
#print(X.columns)

(5952,)
number of classes in target variable  2
number of observations:
 0    5740
1     212
Name: target, dtype: int64


In [7]:
testsize = 0.2  # We are choosing 20% of the data to be test data
seed = 0
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = testsize, random_state = seed)
print('number of observation in training set', len(X_train))
print('number of observation in training set',len(Y_train))

number of observation in training set 4761
number of observation in training set 4761


In [8]:
cat_feat = []
bin_feat = []
num_feat = []

for col in X:
    if ('cat' in col):
        cat_feat.append(col)
    elif ('bin' in col):
        bin_feat.append(col)
    else:
        num_feat.append(col)

# print(cat_feat)
# print(bin_feat)
# print(num_feat)

#print(df.shape)
print("cat feat: ", len(cat_feat))
print("bin feat: ", len(bin_feat))
print("num feat: ", len(num_feat))

cat feat:  14
bin feat:  17
num feat:  26


**Q5) Preprocessing** <font color='purple'> (3) </font>  

Create a `preprocessor` pipeline with the following transformation steps.

(i) "Appropriate" *Imputers* for different types of data (categorical, binary and numerical). <font color='purple'> (1) </font> 

(ii) A *OneHotEncoder* for the categorical features to convert their integer representations to one-hot-vectors. <font color='purple'> (1) </font> 

(iii) A *Scaler* for standardising the resulting data from the above two steps. <font color='purple'> (0.5) </font>

(iv) Print the total number of features after transforming `X_train` with the above constructed `preprocessor`. <font color='purple'> (0.5) </font>


Hint: Use `ColumnTransformer` to consolidate first two steps.

Note: Only the `preprocessor` Pipeline object should be used to `fit` and `transform` the training data. The constituent objects (Imputer, OneHotEncoder and Scaler) should not be separately used to `fit` and `transform`.

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer

# le = LabelEncoder()
# X[cat_feat] = X[cat_feat].apply(lambda col: le.fit_transform(col))
# index = [df.columns.get_loc(c) for c in cat_feat if c in X]
#print(index)
#print(df.dtypes)


categorical_features = cat_feat
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('oneshot',OneHotEncoder(sparse=False, categorical_features = 'all', categories='auto', handle_unknown='ignore'))])


bin_features = bin_feat
bin_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])


numeric_features = num_feat
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])


colTransformer = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', bin_transformer, bin_features)])


preprocessor = Pipeline(steps=[
    ('imputer', colTransformer), 
    ('f_scaler', StandardScaler())])



preprocessor.fit(X_train, Y_train)
preprocessor.transform(X_train)
print(len(X_train.columns))

57


**Q6) Building Different Estimators** <font color='purple'> (6) </font> 

Build four different estimators using the `preprocessor` object from the previous question and the following pipelines of operations:

i. `est1`: Preprocessing -> Classification <font color='purple'> (1.5) </font> 

ii. `est2`: Preprocessing -> Feature Selection -> Classification <font color='purple'> (1.5) </font> 

iii. `est3`: Preprocessing -> Feature Selection -> PCA -> Classification <font color='purple'> (1.5) </font> 

iv. `est4`: Preprocessing -> PCA -> Classification <font color='purple'> (1.5) </font> 

The four estimators should differ in one or more functions chosen for feature selection and/or classification (e.g. you could use [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) and LogisticRegression in `est2`, and [VarianceThreshold](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold) and LogisticRegression in `est3`. 

You are free to use any classifier or [feature selection method](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) from sklearn (not just the ones listed above or discussed in class) that can be used in these pipelines.

Identify at least one hyperparameter per operation in each pipeline. 

For each selected hyperparameter, choose two values -- they may or may not be the same across pipelines. 

Use grid search over the hyperparameter values with 5-fold cross validation and `roc_auc` scoring. 

Evaluate the four best estimators over 5-fold cross validation on the training data. 

Print the mean and std of AUC over the 5-folds for each best estimator. 

<font color='blue'> Among the 4 best estimators found above, which has highest AUC score? </font>

In [18]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import VarianceThreshold

base = 0
est1 = Pipeline(steps=[
    #('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      multi_class='auto'))])
hparam_grid = {'classifier__C': [1., 0.5]}
estimator1 = GridSearchCV(est1, hparam_grid, cv=5, scoring = 'roc_auc')
estimator1.fit(X_train, Y_train)
scores = cross_val_score(estimator1.best_estimator_, X, Y, cv=5, scoring='roc_auc')
print('AUC = {:.3f} mean  +/-  {:.3f} std'.format(scores.mean(), scores.std()))
print(scores.mean(), scores.std())

if (scores.mean() >= base):
    M = estimator1
    base = scores.mean()
    

est2 = Pipeline(steps=[
    #('preprocessor', preprocessor),
    ('selectKBest', SelectKBest(k=10)),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      multi_class='auto'))])
hparam_grid = {'selectKBest__k': [5, 10],
               'classifier__C': [1., 0.5]}
estimator2 = GridSearchCV(est2, hparam_grid, cv=5, scoring = 'roc_auc')
estimator2.fit(X_train, Y_train)
scores = cross_val_score(estimator2.best_estimator_, X, Y, cv=5, scoring='roc_auc')
print('AUC = {:.3f} mean  +/-  {:.3f} std'.format(scores.mean(), scores.std()))
print(scores.mean(), scores.std())

if (scores.mean() >= base):
    M = estimator2 
    base = scores.mean()


est3 = Pipeline(steps=[
    #('preprocessor', preprocessor),
    ('varianceThreshold', VarianceThreshold(threshold=1)),
    ('dim_reducer', PCA(n_components=4)),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      multi_class='auto'))])
hparam_grid = {'varianceThreshold__threshold': [0.1, 0.2],
               'dim_reducer__n_components': [5, 10],
               'classifier__C': [1., 0.5]}
estimator3 = GridSearchCV(est3, hparam_grid, cv=5, scoring = 'roc_auc')
estimator3.fit(X_train, Y_train)
scores = cross_val_score(estimator3.best_estimator_, X, Y, cv=5, scoring='roc_auc')
print('AUC = {:.3f} mean  +/-  {:.3f} std'.format(scores.mean(), scores.std()))
print(scores.mean(), scores.std())

if (scores.mean() >= base):
    M = estimator3 
    base = scores.mean()

    
est4 = Pipeline(steps=[
    #('preprocessor', preprocessor),
    ('dim_reducer', PCA(n_components=4)),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      multi_class='auto'))])
hparam_grid4 = {'dim_reducer__n_components': [5, 10],
               'classifier__C': [1., 0.5]}
estimator4 = GridSearchCV(est4, hparam_grid4, cv=5, scoring = 'roc_auc')
estimator4.fit(X_train, Y_train)
scores = cross_val_score(estimator4.best_estimator_, X, Y, cv=5, scoring='roc_auc')
print('AUC = {:.3f} mean  +/-  {:.3f} std'.format(scores.mean(), scores.std()))
print(scores.mean(), scores.std())


if (scores.mean() >= base):
    M = estimator4
    base = scores.mean()

AUC = 0.552 mean  +/-  0.039 std
0.5516044080706588 0.03875715215749445
AUC = 0.599 mean  +/-  0.054 std
0.5990014894216337 0.05436998871142327
AUC = 0.574 mean  +/-  0.024 std
0.5740572462677641 0.02372507251869663
AUC = 0.574 mean  +/-  0.024 std
0.5743066086332435 0.023708982241985786


**Q7) Class Imbalance** <font color='purple'> (3.5) </font> 

Learn what is class imbalance by reading [this article](https://towardsdatascience.com/handling-imbalanced-datasets-in-machine-learning-7a0e84220f28). 

Additional links to packages for handling class imbalance in python: [Imbalanced-learn.org](http://imbalanced-learn.org/en/stable/index.html), [Over-sampling](http://imbalanced-learn.org/en/stable/over_sampling.html#a-practical-guide), [Under-sampling](http://imbalanced-learn.org/en/stable/under_sampling.html).

Install the `imbalanced-learn` package within Anaconda (See installation details in the link given above). We'll denote the estimator with the best average AUC in the previous question by `M`. 

+ Print the `accuracy_score`, `precision_score`, `recall_score` and `confusion_matrix` of `M` on the test dataset. <font color='purple'> (0.5) </font>  
<font color='blue'> In this case, dose high accuracy score reflect better performance? </font> 

+ Create two more estimators
    1. `M_over`
    2. `M_under`  
    
  by adding 
  
    1. SMOTE 
    2. under-sampler
    
    respectively to the preprocessing pipeline (just before scaling transformation) of M.
    
    Note that for this you may have to use the [Pipeline object](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.pipeline.Pipeline.html) in imbalanced-learn.

    Train both these estimators on the training data. 

    Print the mean and standard deviation of the AUC values, over the 5 folds, for each estimator on the training data.

    Print the accuracy_score, precision, recall and confusion_matrix of `M_over` and `M_under` on the test data. <font color='purple'> (3) </font>  

<font color='blue'> How does SMOTE or under-sampling impact (increase/decrease) the learning performance on training and test data as compared to `M`? </font>

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix

Y_pred = M.predict(X_test)
print("test accuracy = {:.2%}".format(accuracy_score(Y_test, Y_pred)))
print("test precision = {:.2%}".format(precision_score(Y_test, Y_pred)))
print("test recall = {:.2%}".format(recall_score(Y_test, Y_pred)))

cm = confusion_matrix(Y_test, Y_pred)
print("confusion matrix:\n ", cm)

test accuracy = 96.31%
test precision = 0.00%
test recall = 0.00%
confusion matrix:
  [[1147    0]
 [  44    0]]


In [12]:
from imblearn.over_sampling import SMOTE

from imblearn.under_sampling import RandomUnderSampler

# smt = SMOTE()

# preprocessor1 = Pipeline(steps=[
#     ('imputer', colTransformer), 
#     ('smt', smt),
#     ('f_scaler', StandardScaler())])

# preprocessor2 = Pipeline(steps=[
#     ('imputer', colTransformer),
#     ('randomUnderSampling', RandomUnderSampler()),
#     ('f_scaler', StandardScaler())])

#X_resampled, Y_resampled = smt.fit_resample(X_train, Y_train)
# X_resampled, Y_resampled = smt.fit_resample(X, Y)
# preprocessor1.fit(X_train, Y_train)
# preprocessor1.transform(X_train)

# est4 = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('dim_reducer', PCA(n_components=4)),
#     ('classifier', LogisticRegression(solver='lbfgs', 
#                                       multi_class='auto'))])
# hparam_grid4 = {'dim_reducer__n_components': [5, 10],
#                'classifier__C': [1., 0.5]}
# estimator4 = GridSearchCV(est4, hparam_grid4, cv=5, scoring = 'roc_auc')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = testsize, random_state = seed)

In [19]:
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
#from sklearn.pipeline import Pipeline

#print(df.dtypes)
# est4 = Pipeline(steps=[
#     ('preprocessor1', preprocessor1),
#     ('dim_reducer', PCA(n_components=4)),
#     ('classifier', LogisticRegression(solver='lbfgs', 
#                                       multi_class='auto'))])
# hparam_grid4 = {'dim_reducer__n_components': [5, 10],
#                'classifier__C': [1., 0.5]}



M_over = Pipeline(steps=[
    ('smt', SMOTE()),
    ('est', est2)])

# hparam_grid5 = {'dim_reducer__n_components': [5, 10],
#                'classifier__C': [1., 0.5]}

# #print(X)
# M_over = GridSearchCV(est5, hparam_grid5, cv=5, scoring = 'roc_auc')
# a = pd.DataFrame(X_train, columns=X.columns)
# b = pd.DataFrame(Y_train, columns=['target'])

# print(Y_train)
# print(b)
#M_over.fit(a, b)
M_over.fit(X_train, Y_train)


scores = cross_val_score(M_over, X, Y, cv=5, scoring='roc_auc')
print('AUC = {:.3f} mean  +/-  {:.3f} std'.format(scores.mean(), scores.std()))
print(scores.mean(), scores.std())

AUC = 0.559 mean  +/-  0.045 std
0.5590136054421768 0.045314250968312014


In [20]:
#from sklearn.pipeline import Pipeline


#print(df.dtypes)
# est5 = Pipeline(steps=[
#     ('preprocessor2', preprocessor2),
#     ('dim_reducer', PCA(n_components=4)),
#     ('classifier', LogisticRegression(solver='lbfgs', 
#                                       multi_class='auto'))])
# hparam_grid5 = {'dim_reducer__n_components': [5, 10],
#                'classifier__C': [1., 0.5]}

# M_under = GridSearchCV(est5, hparam_grid5, cv=5, scoring = 'roc_auc')
# a = pd.DataFrame(X_resampled, columns=X.columns)
# b = pd.DataFrame(Y_resampled, columns=['target'])
# M_under.fit(a, b)

M_under = Pipeline(steps=[
    ('under', RandomUnderSampler()),
    ('est4', est2)])
M_under.fit(X_train, Y_train)
scores = cross_val_score(M_under, X, Y, cv=5, scoring='roc_auc')
print('AUC = {:.3f} mean  +/-  {:.3f} std'.format(scores.mean(), scores.std()))
print(scores.mean(), scores.std())

AUC = 0.541 mean  +/-  0.027 std
0.5410282604249869 0.0266091520822533


**Q8) Ensemble Method**  <font color='purple'> (1) </font> 

To answer this question, learn about Random Forest Classifier. You may follow these links:
[Understanding Random Forest](https://towardsdatascience.com/understanding-random-forest-58381e0602d2), and [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). 

+ Create a pipeline with the `preprocessor` object of Q5 and a RandomForest classifier having 100 estimators and a maximum depth of 3. Train it on the training set. 

    Print the `accuracy_sore`, precision, recall and `confusion_matrix` of this model on the test data. <font color='purple'> (1) </font>

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = testsize, random_state = 0)


est1 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0))])


est1.fit(X_train, Y_train)
Y_pred = est1.predict(X_test)
print("Confusion matrix:")
print(confusion_matrix(Y_test, Y_pred))
print("Accuracy = {:.2%}".format(accuracy_score(Y_test, Y_pred)))
print('Precision:', precision_score(Y_test, Y_pred))
print('Recall:', recall_score(Y_test, Y_pred))

Confusion matrix:
[[1147    0]
 [  44    0]]
Accuracy = 96.31%
Precision: 0.0
Recall: 0.0
